In [2]:
from constants.abs_paths import AbsDirPath
import pandas as pd

In [3]:
file_dir = AbsDirPath.KEYWORDS_MATCHING

def load_all_files(dir):
    dfs = []
    for file_path in dir.glob("*.parquet"):
        try:
            file = pd.read_parquet(file_path)
            # file['fname'] = file_path
            dfs.append(file)
            print(f"Loaded {file_path}")
        except:
            print(f"Error while loading {file_path}")

    df = pd.concat(dfs)
    return df


df = load_all_files(file_dir)

Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\matched\allenai.scispacy.v0.5.5.code_comment.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\matched\allenai.scispacy.v0.5.5.docs.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\matched\allenai.scispacy.v0.5.5.issue.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\matched\allenai.scispacy.v0.5.5.issue_comment.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\matched\allenai.scispacy.v0.5.5.release.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\matched\google.deepvariant.v1.6.1.code_comment.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\matched\google.deepvariant.v1.6.1.docs.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis

In [98]:
from IPython.display import display

def display_and_save(df, path):
    df.to_excel(path, merge_cells=False)
    display(df)

def display_and_save_csv(df, path):
    df.to_csv(path)
    display(df)

In [92]:
df.head()

,keyword,keyword_raw,matched_word,match_idx,sentence,qa,source,url,repo_id,sent_kw_n
0,releas,releas,release,28,Instructions for creating a release of the sci...,deployability,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,1
1,version,version,versions,150,Instructions for creating a release of the sci...,deployability,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,6
2,version,version,version,172,Instructions for creating a release of the sci...,deployability,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,6
3,instal,instal,install,297,Instructions for creating a release of the sci...,deployability,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,1
4,version,version,versions,150,Instructions for creating a release of the sci...,integrability,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,6


In [10]:
df.shape

(1994850, 9)

In [34]:
display_and_save(df.pivot_table(index=["source"], columns=["qa"], values="keyword", aggfunc="count", fill_value=0, margins=True).sort_values(by="All", ascending=False), AbsDirPath.RES_KEYWORDS_MATCHING / "pivot_source_to_qa.xlsx")

qa,availability,deployability,energy efficiency,integrability,interoperability,modifiability,performance,reliability,safety,security,testability,usability,All
source,,,,,,,,,,,,,
All,108717,249773,138534,178433,144357,164426,165049,44260,242789,171377,145220,241915,1994850
code_comment,60524,103522,76213,106345,85125,94150,87751,22887,127218,106838,69233,120385,1060191
issue_comment,17477,62070,22141,23041,17370,22784,27806,9865,45907,21973,29418,40884,340736
issue,17206,48990,21653,23323,15490,23610,26801,6879,42181,20499,24024,40459,311115
docs,12398,29488,17734,23385,25120,22477,21741,4289,25917,20131,21148,36670,260498
wiki,1069,5446,715,2263,1189,1334,872,317,1450,1811,1360,3373,21199
release,43,257,78,76,63,71,78,23,116,125,37,144,1111


In [100]:
display_and_save(df.pivot_table(index=["source", "repo_id"], columns=["qa"], values="keyword", aggfunc="count", fill_value=0, margins=True), AbsDirPath.RES_KEYWORDS_MATCHING / "pivot_source&repo_to_qa.xlsx")

qa                                        availability  deployability  \
source        repo_id                                                   
code_comment  OpenGene/fastp/v0.23.4               233            186   
              allenai/scispacy/v0.5.5                9             41   
              google/deepvariant/v1.6.1           1141           1385   
              root-project/root/v6-32-06         58825         101045   
              scverse/scanpy/1.10.2                316            865   
docs          OpenGene/fastp/v0.23.4                27             60   
              allenai/scispacy/v0.5.5               41            127   
              google/deepvariant/v1.6.1            168            353   
              root-project/root/v6-32-06         12088          28545   
              scverse/scanpy/1.10.2                 74            403   
issue         OpenGene/fastp/v0.23.4               396            588   
              allenai/scispacy/v0.5.5              619           2017   
              google/deepvariant/v1.6.1           2186           3803   
              root-project/root/v6-32-06         10785          34143   
              scverse/scanpy/1.10.2               3220           8439   
issue_comment OpenGene/fastp/v0.23.4               295            810   
              allenai/scispacy/v0.5.5              465           1459   
              google/deepvariant/v1.6.1           2645           7313   
              root-project/root/v6-32-06         10202          42380   
              scverse/scanpy/1.10.2               3870          10108   
release       OpenGene/fastp/v0.23.4                 2              3   
              allenai/scispacy/v0.5.5                1             45   
              google/deepvariant/v1.6.1             28             76   
              root-project/root/v6-32-06            12            133   
wiki          root-project/root/v6-32-06           528           1650   
              scverse/scanpy/1.10.2                541           3796   
All                                             108717         249773   

qa                                        energy efficiency  integrability  \
source        repo_id                                                        
code_comment  OpenGene/fastp/v0.23.4                    144            494   
              allenai/scispacy/v0.5.5                    23             51   
              google/deepvariant/v1.6.1                 583           2273   
              root-project/root/v6-32-06              75080         102847   
              scverse/scanpy/1.10.2                     383            680   
docs          OpenGene/fastp/v0.23.4                     51            118   
              allenai/scispacy/v0.5.5                    50             70   
              google/deepvariant/v1.6.1                 310            177   
              root-project/root/v6-32-06              17251          22815   
              scverse/scanpy/1.10.2                      72            205   
issue         OpenGene/fastp/v0.23.4                    698           1137   
              allenai/scispacy/v0.5.5                   664            858   
              google/deepvariant/v1.6.1                2102           2566   
              root-project/root/v6-32-06              15861          14086   
              scverse/scanpy/1.10.2                    2328           4676   
issue_comment OpenGene/fastp/v0.23.4                    496            923   
              allenai/scispacy/v0.5.5                   746            760   
              google/deepvariant/v1.6.1                3444           3361   
              root-project/root/v6-32-06              14668          12112   
              scverse/scanpy/1.10.2                    2787           5885   
release       OpenGene/fastp/v0.23.4                      5              7   
              allenai/scispacy/v0.5.5                    10             19   
          

In [36]:
df.groupby(["sentence", "source", "keyword"])['matched_word'].count().sort_values(ascending=False)

sentence                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [73]:
df["sent_kw_n"] = df.groupby(["sentence", "keyword"]).transform('size')

,,keyword_raw,matched_word,match_idx,qa,source,url,repo_id,sent_kw_n
sentence,keyword,,,,,,,,
@phsft-bot build,build,build,build,11,deployability,issue_comment,https://github.com/root-project/root/pull/121,root-project/root/v6-32-06,2314
@phsft-bot build!,build,build,build,11,deployability,issue_comment,https://github.com/root-project/root/pull/94,root-project/root/v6-32-06,1124
"Copyright 2018 Google LLC.\r \r Redistribution and use in source and binary forms, with or without\r modification, are permitted provided that the following conditions\r are met:\r \r 1. Redistributions of source code must retain the above copyright notice,\r this list of conditions and the following disclaimer.\r \r 2. Redistributions in binary form must reproduce the above copyright\r notice, this list of conditions and the following disclaimer in the\r documentation and/or other materials provided with the distribution.\r \r 3. Neither the name of the copyright holder nor the names of its\r contributors may be used to endorse or promote products derived from this\r software without specific prior written permission.\r \r THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS ""AS IS""\r AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE\r IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE\r ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE\r LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR\r CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF\r SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS\r INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN\r CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE)\r ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE\r POSSIBILITY OF SUCH DAMAGE.",servic,servic,SERVICES,1179,availability,code_comment,https://github.com/google/deepvariant/tree/v1....,google/deepvariant/v1.6.1,564
MENU*,menu,menu,MENU,0,usability,code_comment,https://github.com/root-project/root/tree/v6-3...,root-project/root/v6-32-06,541
@phsft-bot build with flags -DCTEST_TEST_EXCLUDE_NONE=On,build,build,build,11,deployability,issue_comment,https://github.com/root-project/root/pull/1306,root-project/root/v6-32-06,506
...,...,...,...,...,...,...,...,...,...
"Adapter cutting metrics not output for single end reads when an adapters FASTA is used; I have observed that giving an adapter FASTA removes the `adapter_cutting` section from the output. I'd like to know which adapter(s) were used in trimming. ~Similarly if I auto-detect~. Here's a simple test case to reproduce:. ```. $ fastp --version. fastp 0.20.1. $ cat in.fastq . @readname. TNGGAGCTGTTTATACTATAATCAAATACACTGTAGGCACCATCAATAGAT. +. ###################################################. $ cat adapters.fasta . >some_adapter. CTGTAGGCACCATCAAT. $$ fastp --in1 in.fastq --out1 out.fastq --json out.json --adapter_fasta adapters.fasta. Detecting adapter sequence for read1... No adapter detected for read1. Read1 before filtering:. total reads: 1. total bases: 51. Q20 bases: 0(0%). Q30 bases: 0(0%). Read1 after filtering:. total reads: 0. total bases: 0. Q20 bases: 0(-nan%). Q30 bases: 0(-nan%). Filtering result:. reads passed filter: 0. reads failed due to low quality: 1. reads failed due to too many N: 0. reads failed due to too short: 0. reads with adapter trimmed: 1. bases trimmed due to adapters: 21. Duplication rate (may be overestimated since this is SE data): 0%. JSON report: out.json. HTML report: fastp.html. fastp --in1 in.fastq --out1 out.fastq --json out.json --adapter_fasta adapters.fasta . fastp v0.20.1, time used: 0 seconds. $ grep adapter_cutting out.json . $ echo $? 1. ```. So the output reports we trimmed the read but no JSON output of which adapter was used. But then if I run it specifying a single adapter, we get a report:. ```. $ fastp --in1 in.fastq --out1 out.fastq --json out.j

In [99]:
display_and_save_csv(df[df.sent_kw_n > 50].groupby(["sentence", "keyword"]).first().sort_values(by="sent_kw_n", ascending=False).reset_index(), AbsDirPath.RES_KEYWORDS_MATCHING / "top_duplicate_sentence&kw.csv")

,sentence,keyword,keyword_raw,matched_word,match_idx,qa,source,url,repo_id,sent_kw_n
0,@phsft-bot build,build,build,build,11,deployability,issue_comment,https://github.com/root-project/root/pull/121,root-project/root/v6-32-06,2314
1,@phsft-bot build!,build,build,build,11,deployability,issue_comment,https://github.com/root-project/root/pull/94,root-project/root/v6-32-06,1124
2,Copyright 2018 Google LLC.\r \r Redistribution...,servic,servic,SERVICES,1179,availability,code_comment,https://github.com/google/deepvariant/tree/v1....,google/deepvariant/v1.6.1,564
3,MENU*,menu,menu,MENU,0,usability,code_comment,https://github.com/root-project/root/tree/v6-3...,root-project/root/v6-32-06,541
4,@phsft-bot build with flags -DCTEST_TEST_EXCLU...,build,build,build,11,deployability,issue_comment,https://github.com/root-project/root/pull/1306,root-project/root/v6-32-06,506
...,...,...,...,...,...,...,...,...,...,...
290,Adapter cutting metrics not output for single ...,adapt,adapt,Adapter,0,energy efficiency,issue,https://github.com/OpenGene/fastp/issues/269,OpenGene/fastp/v0.23.4,52
291,olcache/Python/3.8.8/x64/lib/python3.8/site-pa...,log,log,logging,4506,deployability,issue,https://github.com/scverse/scanpy/issues/1736,scverse/scanpy/1.10.2,52
292,Look up the name in the global module table.,modul,modul,module,31,deployability,code_comment,https://github.com/root-project/root/tree/v6-3...,root-project/root/v6-32-06,51
293,Pass Pipeline Configuration,configur,configur,Configuration,14,integrability,code_comment,https://github.com/root-project/root/tree/v6-3...,root-project/root/v6-32-06,51


In [62]:
df.groupby(["sentence", "keyword"]).size().rename("size")[lambda x: x > 100].sort_values(ascending=False).first()

TypeError: NDFrame.first() missing 1 required positional argument: 'offset'

In [65]:
df[df.groupby(["sentence", "keyword"])['matched_word'].transform('count') > 100].groupby(["sentence", "keyword"]).first()

,,keyword_raw,matched_word,match_idx,qa,source,url,repo_id
sentence,keyword,,,,,,,
scanpy.Neighbors.compute_neighbors. scanpy.Neighbors.compute_transitions. scanpy.Neighbors.getdoc. scanpy.Neighbors.to_igraph. Settings. scanpy.set_figure_params. scanpy._settings.ScanpyConfig. scanpy._settings.ScanpyConfig.autosave. scanpy._settings.ScanpyConfig.autoshow. scanpy._settings.ScanpyConfig.cache_compression. scanpy._settings.ScanpyConfig.cachedir. scanpy._settings.ScanpyConfig.categories_to_ignore. scanpy._settings.ScanpyConfig.datasetdir. scanpy._settings.ScanpyConfig.figdir. scanpy._settings.ScanpyConfig.file_format_data. scanpy._settings.ScanpyConfig.file_format_figs. scanpy._settings.ScanpyConfig.logfile. scanpy._settings.ScanpyConfig.logpath. scanpy._settings.ScanpyConfig.max_memory. scanpy._settings.ScanpyConfig.n_jobs. scanpy._settings.ScanpyConfig.plot_suffix. scanpy._settings.ScanpyConfig.verbosity. scanpy._settings.ScanpyConfig.writedir. scanpy._settings.ScanpyConfig.N_PCS. scanpy._settings.ScanpyConfig.set_figure_params. scanpy.logging.print_header. scanpy.logging.print_versions. Datasets. scanpy.datasets.blobs. scanpy.datasets.ebi_expression_atlas. scanpy.datasets.krumsiek11. scanpy.datasets.moignard15. scanpy.datasets.pbmc3k. scanpy.datasets.pbmc3k_processed. scanpy.datasets.pbmc68k_reduced. scanpy.datasets.paul15. scanpy.datasets.toggleswitch. scanpy.datasets.visium_sge. Deprecated functions. scanpy.pp.filter_genes_dispersion. scanpy.pp.normalize_per_cell. External API. Preprocessing: PP. scanpy.external.pp.bbknn. scanpy.external.pp.harmony_integrate. scanpy.external.pp.mnn_correct. scanpy.external.pp.scanorama_integrate. scanpy.external.pp.hashsolo. scanpy.external.pp.dca. scanpy.external.pp.magic. Tools: TL. scanpy.external.tl.phate. scanpy.external.tl.palantir. scanpy.external.tl.trimap. scanpy.external.tl.sam. scanpy.external.tl.phenograph. scanpy.external.tl.harmony_timeseries. scanpy.external.tl.wishbone. scanpy.external.tl.palantir. scanpy.external.tl.palantir_results. scanpy.external.tl.sandbag. scanpy.external.tl.cyclone. Plottin,log,log,logging,7805,deployability,wiki,https://scanpy.readthedocs.io/en/stable/commun...,scverse/scanpy/1.10.2
1.2.2 OpenMP Language Terminology Structured block - An executable statement with a single entry at the top and a single exit at the bottom. The point of exit cannot be a branch out of the structured block. longjmp() and throw() must not violate the entry/exit criteria.,state,state,statement,67,availability,code_comment,https://github.com/root-project/root/tree/v6-3...,root-project/root/v6-32-06
<! message logger,log,log,logger,11,deployability,code_comment,https://github.com/root-project/root/tree/v6-3...,root-project/root/v6-32-06
"@(#)root/eve:$Id$\r Authors: Matevz Tadel & Alja Mrak-Tadel: 2006, 2007\r \r Copyright (C) 1995-2007, Rene Brun and Fons Rademakers. *\r All rights reserved. *\r \r For the licensing terms see $ROOTSYS/LICENSE. *\r For the list of contributors see $ROOTSYS/README/CREDITS. *",auth,auth,Authors,19,security,code_comment,https://github.com/root-project/root/tree/v6-3...,root-project/root/v6-32-06
"@(#)root/minuit2:$Id$\r Authors: M. Winkler, F. James, L. Moneta, A. Zsenei 2003-2005\r \r \r Copyright (c) 2005 LCG ROOT Math team, CERN/PH-SFT *",auth,auth,Authors,23,security,code_comment,https://github.com/root-project/root/tree/v6-3...,root-project/root/v6-32-06
...,...,...,...,...,...,...,...,...
l.pp.recipe_pearson_residuals. Classes. scanpy.Neighbors. scanpy.Neighbors.connectivities. scanpy.Neighbors.distances. scanpy.Neighbors.distances_dpt. scanpy.Neighbors.eigen_basis. scanpy.Neighbors.eigen_values. scanpy.Neighbors.rp_forest. scanpy.Neighbors.transitions. scanpy.Neighbors.transitions_sym. scanpy.Neighbors.compute_eigen. scanpy.Neighbors.compute_neighbors. scanpy.Neighbors.compute_transitions. scanpy.Neighbors.getdoc. scanpy.Neighbors.to_igraph. Settings. scanpy.set_figure_params. scanpy._settings.ScanpyConfig. scanpy._settings.ScanpyConfig.auto

In [58]:
df[df.groupby(["sentence", "keyword"]).transform('size') > 100]

,keyword,keyword_raw,matched_word,match_idx,sentence,qa,source,url,repo_id
0,incid,incid,INCIDENTAL,1053,Copyright 2017 Google LLC.\r \r Redistribution...,availability,code_comment,https://github.com/google/deepvariant/tree/v1....,google/deepvariant/v1.6.1
1,servic,servic,SERVICES,1179,Copyright 2017 Google LLC.\r \r Redistribution...,availability,code_comment,https://github.com/google/deepvariant/tree/v1....,google/deepvariant/v1.6.1
2,servic,servic,SERVICES,1179,Copyright 2017 Google LLC.\r \r Redistribution...,deployability,code_comment,https://github.com/google/deepvariant/tree/v1....,google/deepvariant/v1.6.1
5,servic,servic,SERVICES,1179,Copyright 2017 Google LLC.\r \r Redistribution...,integrability,code_comment,https://github.com/google/deepvariant/tree/v1....,google/deepvariant/v1.6.1
6,contract,contract,CONTRACT,1306,Copyright 2017 Google LLC.\r \r Redistribution...,integrability,code_comment,https://github.com/google/deepvariant/tree/v1....,google/deepvariant/v1.6.1
...,...,...,...,...,...,...,...,...,...
11618,log,log,logging,7843,scanpy.Neighbors.compute_neighbors. scanpy.Ne...,security,wiki,https://scanpy.readthedocs.io/en/stable/tutori...,scverse/scanpy/1.10.2
11631,log,log,logfile,7469,ghly_variable_genes. scanpy.experimental.pp.re...,testability,wiki,https://scanpy.readthedocs.io/en/stable/tutori...,scverse/scanpy/1.10.2
11632,log,log,logpath,7508,l.pp.recipe_pearson_residuals. Classes. scanpy...,testability,wiki,https://scanpy.readthedocs.io/en/stable/tutori...,scverse/scanpy/1.10.2
11633,log,log,logging,7814,anpy.Neighbors.compute_eigen. scanpy.Neighbors...,testability,wiki,https://scanpy.readthedocs.io/en/stable/tutori...,scverse/scanpy/1.10.2


In [63]:
df.groupby(["sentence", "keyword"]).first()

keyword_raw  \
sentence                                           keyword                  
\t\t\t\tCOMMAND ${CMAKE_COMMAND} -E copy_if_dif... build            build   
\t\t const char *__restrict __src, size_t __n).... build            build   
\t .globl\ttest1. \t .align\t16, 0x90. \t pushq... format          format   
\t%rbp'. [003] {Code} 'retq'. [003] 9 {Line}. [... log                log   
\t%xmm3, %xmm3, %xmm4. [2,0] . DeeE-----R . vmu... time              time   
...                                                                   ...   
🤔 Just to understand, is it a known real leak o... understand  understand   
                                                   unit              unit   
🥇 great. About the c++14 standard: does this me... automat        automat   
                                                   standard      standard   
                                                   user              user   

                                                                matched_word  \
sentence                                           keyword                     
\t\t\t\tCOMMAND ${CMAKE_COMMAND} -E copy_if_dif... build            Building   
\t\t const char *__restrict __src, size_t __n).... build            builddir   
\t .globl\ttest1. \t .align\t16, 0x90. \t pushq... format             Format   
\t%rbp'. [003] {Code} 'retq'. [003] 9 {Line}. [... log               Logical   
\t%xmm3, %xmm3, %xmm4. [2,0] . DeeE-----R . vmu... time                times   
...                                                                      ...   
🤔 Just to understand, is it a known real leak o... understand     understand   
                                                   unit                 unit   
🥇 great. About the c++14 standard: does this me... automat     automatically   
                                                   standard         standard   
                                                   user                users   

                                                               match_idx  \
sentence                                           keyword                 
\t\t\t\tCOMMAND ${CMAKE_COMMAND} -E copy_if_dif... build            3447   
\t\t const char *__restrict __src, size_t __n).... build            1698   
\t .globl\ttest1. \t .align\t16, 0x90. \t pushq... format          19967   
\t%rbp'. [003] {Code} 'retq'. [003] 9 {Line}. [... log             27226   
\t%xmm3, %xmm3, %xmm4. [2,0] . DeeE-----R . vmu... time            23276   
...                                                                  ...   
🤔 Just to understand, is it a known real leak o... understand         10   
                                                   unit              774   
🥇 great. About the c++14 standard: does this me... automat           114   
                                                   standard           25   
                                                   user              192   

                                                                             qa  \
sentence                                           keyword                        
\t\t\t\tCOMMAND ${CMAKE_COMMAND} -E copy_if_dif... build          deployability   
\t\t const char *__restrict __src, size_t __n).... build          deployability   
\t .globl\ttest1. \t .align\t16, 0x90. \t pushq... format      interoperability   
\t%rbp'. [003] {Code} 'retq'. [003] 9 {Line}. [... log            deployability   
\t%xmm3, %xmm3, %xmm4. [2,0] . DeeE-----R . vmu... time             performance   
...                                                                         ...   
🤔 Just to understand, is it a known real leak o... understand       testability   
                                                   unit             testability   
🥇 great. About the c++14 standard: does this me... automat        deployability   
                                                   standard    interoperability   
                                  

In [10]:
df.groupby(["sentence", "source", "keyword", "url"])['id'].count().sort_values(ascending=False)

sentence                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [11]:
df['sentence_counts'] = df.groupby(["sentence", "source", "keyword"])['id'].transform('count')

In [13]:
import hashlib

quick_hash = lambda x: hashlib.md5(x.encode('utf-8')).hexdigest()

df['text_hash'] = df['sentence'].apply(quick_hash)

In [14]:
df['real_id'] = df.repo_id + "." + df['source'] + "." + df['qa'] + "." +  df['text_hash'] + "." + df['keyword'].str.lower() + "." +df['matched_word'].str.lower()

In [15]:
df.groupby(["real_id"])['id'].count().sort_values(ascending=False)

real_id
root.root-project.ISSUE_COMMENT.deployability.b94f3fb06d8d249282128e958f0d0907.build.build      2314
root.root-project.ISSUE_COMMENT.deployability.fd35b23a68240e34b4f2299982b546e5.build.build      1124
root.root-project.ISSUE_COMMENT.deployability.15145c695b97fb36624634b9efdd107a.build.build       506
root.root-project.CODE_COMMENT.usability.18a558d0840ceff6fd9f5660c2d9e721.menu.menu              497
root.root-project.CODE_COMMENT.usability.486d27f6f919a30a53cff74832b88164.prototyp.prototype     411
                                                                                                ... 
scispacy.allenai.RELEASES.security.b094573ea932611f669adc375eb58463.model.models                   1
scispacy.allenai.RELEASES.security.ae418592347841c6e80ec9e5497b98d6.model.models                   1
scispacy.allenai.RELEASES.security.991da59c5775c9a3295c05b2a05b2f77.updat.updated                  1
scispacy.allenai.RELEASES.security.991da59c5775c9a3295c05b2a05b2f77.modif.modify   

In [16]:
new_dataset_count = df.groupby(["real_id"])['id'].count().sort_values(ascending=False).count()
new_dataset_count

np.int64(1612263)

In [17]:
old_count = df['id'].count()
old_count

np.int64(1997126)

In [18]:
new_dataset_count / old_count

np.float64(0.8072915779975826)

In [19]:
df['similar_count'] = df.groupby(["real_id"])['id'].transform('count')

In [20]:
df = df.groupby(['real_id']).first()

In [21]:
df.head()

,id,quality_attribute,keyword,matched_word,match_idx,sentence,source,filename,author,repo,version,wiki,url,fname,sentence_counts,text_hash,similar_count
real_id,,,,,,,,,,,,,,,,,
deepvariant.google.CODE_COMMENT.availability.00a64ba49a8c8077515638fc173723bc.checkpoint.checkpoint,https://github.com/google/deepvariant/tree/v1....,availability,checkpoint,checkpoint,32.0,"""""""Return the path to the model checkpoint bas...",CODE_COMMENT,scripts/run_deepsomatic.py,google,deepvariant,v1.6.1,None,https://github.com/google/deepvariant/tree/v1....,C:\Users\Dmitrijs\Documents\myDocs\masters\cou...,6,00a64ba49a8c8077515638fc173723bc,3
deepvariant.google.CODE_COMMENT.availability.00bdf0c917332338e88e526ef0912d47.checkpoint.checkpoint,https://github.com/google/deepvariant/tree/v1....,availability,checkpoint,checkpoint,49.0,# If the class variable name doesn't existin t...,CODE_COMMENT,deepvariant/dv_utils_test.py,google,deepvariant,v1.6.1,None,https://github.com/google/deepvariant/tree/v1....,C:\Users\Dmitrijs\Documents\myDocs\masters\cou...,2,00bdf0c917332338e88e526ef0912d47,1
deepvariant.google.CODE_COMMENT.availability.00c0e7cdceae45fdecf2163aa50041d2.incid.incidental,https://github.com/google/deepvariant/tree/v1....,availability,incid,INCIDENTAL,1092.0,/*. * Copyright 2018 Google LLC. *. * Redistri...,CODE_COMMENT,deepvariant/realigner/fast_pass_aligner.cc,google,deepvariant,v1.6.1,None,https://github.com/google/deepvariant/tree/v1....,C:\Users\Dmitrijs\Documents\myDocs\masters\cou...,46,00c0e7cdceae45fdecf2163aa50041d2,23
deepvariant.google.CODE_COMMENT.availability.00c0e7cdceae45fdecf2163aa50041d2.servic.services,https://github.com/google/deepvariant/tree/v1....,availability,servic,SERVICES,1222.0,/*. * Copyright 2018 Google LLC. *. * Redistri...,CODE_COMMENT,deepvariant/realigner/fast_pass_aligner.cc,google,deepvariant,v1.6.1,None,https://github.com/google/deepvariant/tree/v1....,C:\Users\Dmitrijs\Documents\myDocs\masters\cou...,92,00c0e7cdceae45fdecf2163aa50041d2,23
deepvariant.google.CODE_COMMENT.availability.00d2b41f72e591d66c3134b8110fdd4f.operat.operator,https://github.com/google/deepvariant/tree/v1....,availability,operat,operator,12.0,# The match operator = indicates that a positi...,CODE_COMMENT,deepvariant/realigner/window_selector_test.py,google,deepvariant,v1.6.1,None,https://github.com/google/deepvariant/tree/v1....,C:\Users\Dmitrijs\Documents\myDocs\masters\cou...,1,00d2b41f72e591d66c3134b8110fdd4f,1


In [22]:
df[df.sentence.str.startswith("@phsft-bot build")]

,id,quality_attribute,keyword,matched_word,match_idx,sentence,source,filename,author,repo,version,wiki,url,fname,sentence_counts,text_hash,similar_count
real_id,,,,,,,,,,,,,,,,,
root.root-project.ISSUE_COMMENT.availability.01478f2e78b954d907834f1a8637e635.error.errored,https://github.com/root-project/root/pull/8694:52,availability,error,errored,52.0,@phsft-bot build just on mac11.0/cxx17. I thin...,ISSUE_COMMENT,None,root-project,root,v6-32-06,https://root.cern,https://github.com/root-project/root/pull/8694,C:\Users\Dmitrijs\Documents\myDocs\masters\cou...,4,01478f2e78b954d907834f1a8637e635,1
root.root-project.ISSUE_COMMENT.availability.08afaec1a2cfa0fb59e039f432081840.state.state,https://github.com/root-project/root/pull/2128:51,availability,state,state,51.0,"@phsft-bot build! @Axel-Naumann, IIUC, the cur...",ISSUE_COMMENT,None,root-project,root,v6-32-06,https://root.cern,https://github.com/root-project/root/pull/2128,C:\Users\Dmitrijs\Documents\myDocs\masters\cou...,2,08afaec1a2cfa0fb59e039f432081840,1
root.root-project.ISSUE_COMMENT.availability.09e167f26fc9f527bf7e6c92c34998d4.error.error,https://github.com/root-project/root/pull/6877:80,availability,error,error,80.0,@phsft-bot build just on ROOT-performance-cent...,ISSUE_COMMENT,None,root-project,root,v6-32-06,https://root.cern,https://github.com/root-project/root/pull/6877,C:\Users\Dmitrijs\Documents\myDocs\masters\cou...,4,09e167f26fc9f527bf7e6c92c34998d4,1
root.root-project.ISSUE_COMMENT.availability.0df13ff971742d57d66f3d43fc5075ee.failur.failures,https://github.com/root-project/root/pull/8176:39,availability,failur,failures,39.0,@phsft-bot build. I can't see any test failure...,ISSUE_COMMENT,None,root-project,root,v6-32-06,https://root.cern,https://github.com/root-project/root/pull/8176,C:\Users\Dmitrijs\Documents\myDocs\masters\cou...,2,0df13ff971742d57d66f3d43fc5075ee,1
root.root-project.ISSUE_COMMENT.availability.0f28511d1518ff218b7b0d7c855f8b10.error.errors,https://github.com/root-project/root/pull/7910:40,availability,error,errors,40.0,@phsft-bot build! To see if the Windows errors...,ISSUE_COMMENT,None,root-project,root,v6-32-06,https://root.cern,https://github.com/root-project/root/pull/7910,C:\Users\Dmitrijs\Documents\myDocs\masters\cou...,4,0f28511d1518ff218b7b0d7c855f8b10,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
root.root-project.ISSUE_COMMENT.usability.f219b769cd71a5a48cc180bb0cac149e.perform.performance-,https://github.com/root-project/root/pull/5842:30,usability,perform,performance-,30.0,@phsft-bot build just on ROOT-performance-cent...,ISSUE_COMMENT,None,root-project,root,v6-32-06,https://root.cern,https://github.com/root-project/root/pull/5842,C:\Users\Dmitrijs\Documents\myDocs\masters\cou...,24,f219b769cd71a5a48cc180bb0cac149e,12
root.root-project.ISSUE_COMMENT.usability.f227dd94a518345fd61c3b433bc39b18.perform.performance-,https://github.com/root-project/root/pull/3100:41,usability,perform,performance-,41.0,@phsft-bot build with flags just on ROOT-perfo...,ISSUE_COMMENT,None,root-project,root,v6-32-06,https://root.cern,https://github.com/root-project/root/pull/3100,C:\Users\Dmitrijs\Documents\myDocs\masters\cou...,4,f227dd94a518345fd61c3b433bc39b18,2
root.root-project.ISSUE_COMMENT.usability.f2f1759c32a665c308ac447c2ca13cf4.stop.stop,https://github.com/root-project/root/pull/767:21,usability,stop,stop,21.0,@phsft-bot build and stop being lazy,ISSUE_COMMENT,None,root-project,root,v6-32-06,https://root.cern,https://github.com/root-project/root/pull/767,C:\Users\Dmitrijs\Documents\myDocs\masters\cou...,1,f2f1759c32a665c308ac447c2ca13cf4,1


In [23]:
df[df.sentence.str.startswith("Build failed")]

,id,quality_attribute,keyword,matched_word,match_idx,sentence,source,filename,author,repo,version,wiki,url,fname,sentence_counts,text_hash,similar_count
real_id,,,,,,,,,,,,,,,,,
root.root-project.ISSUE_COMMENT.availability.237c3105f27dc32284a9c5e910ca6d8f.error.error,https://github.com/root-project/root/pull/4799...,availability,error,Error,425.0,Build failed with:. ```. 11:40:26 make[2]: ***...,ISSUE_COMMENT,None,root-project,root,v6-32-06,https://root.cern,https://github.com/root-project/root/pull/4799,C:\Users\Dmitrijs\Documents\myDocs\masters\cou...,4,237c3105f27dc32284a9c5e910ca6d8f,1
root.root-project.ISSUE_COMMENT.deployability.237c3105f27dc32284a9c5e910ca6d8f.build.build,https://github.com/root-project/root/pull/4799:0,deployability,build,Build,0.0,Build failed with:. ```. 11:40:26 make[2]: ***...,ISSUE_COMMENT,None,root-project,root,v6-32-06,https://root.cern,https://github.com/root-project/root/pull/4799,C:\Users\Dmitrijs\Documents\myDocs\masters\cou...,3,237c3105f27dc32284a9c5e910ca6d8f,3
root.root-project.ISSUE_COMMENT.deployability.237c3105f27dc32284a9c5e910ca6d8f.fail.failed,https://github.com/root-project/root/pull/4799:6,deployability,fail,failed,6.0,Build failed with:. ```. 11:40:26 make[2]: ***...,ISSUE_COMMENT,None,root-project,root,v6-32-06,https://root.cern,https://github.com/root-project/root/pull/4799,C:\Users\Dmitrijs\Documents\myDocs\masters\cou...,4,237c3105f27dc32284a9c5e910ca6d8f,2
root.root-project.ISSUE_COMMENT.deployability.6bd061ccff03bbbb515b4d8afde33bde.build.build,https://github.com/root-project/root/pull/4512:0,deployability,build,Build,0.0,Build failed because of `Build failed because:...,ISSUE_COMMENT,None,root-project,root,v6-32-06,https://root.cern,https://github.com/root-project/root/pull/4512,C:\Users\Dmitrijs\Documents\myDocs\masters\cou...,2,6bd061ccff03bbbb515b4d8afde33bde,2
root.root-project.ISSUE_COMMENT.deployability.6bd061ccff03bbbb515b4d8afde33bde.fail.failed,https://github.com/root-project/root/pull/4512:6,deployability,fail,failed,6.0,Build failed because of `Build failed because:...,ISSUE_COMMENT,None,root-project,root,v6-32-06,https://root.cern,https://github.com/root-project/root/pull/4512,C:\Users\Dmitrijs\Documents\myDocs\masters\cou...,4,6bd061ccff03bbbb515b4d8afde33bde,2
root.root-project.ISSUE_COMMENT.interoperability.6bd061ccff03bbbb515b4d8afde33bde.plug.plugins,https://github.com/root-project/root/pull/4512:61,interoperability,plug,plugins,61.0,Build failed because of `Build failed because:...,ISSUE_COMMENT,None,root-project,root,v6-32-06,https://root.cern,https://github.com/root-project/root/pull/4512,C:\Users\Dmitrijs\Documents\myDocs\masters\cou...,1,6bd061ccff03bbbb515b4d8afde33bde,1
root.root-project.ISSUE_COMMENT.performance.237c3105f27dc32284a9c5e910ca6d8f.error.error,https://github.com/root-project/root/pull/4799...,performance,error,Error,425.0,Build failed with:. ```. 11:40:26 make[2]: ***...,ISSUE_COMMENT,None,root-project,root,v6-32-06,https://root.cern,https://github.com/root-project/root/pull/4799,C:\Users\Dmitrijs\Documents\myDocs\masters\cou...,4,237c3105f27dc32284a9c5e910ca6d8f,1
root.root-project.ISSUE_COMMENT.reliability.237c3105f27dc32284a9c5e910ca6d8f.fail.failed,https://github.com/root-project/root/pull/4799:6,reliability,fail,failed,6.0,Build failed with:. ```. 11:40:26 make[2]: ***...,ISSUE_COMMENT,None,root-project,root,v6-32-06,https://root.cern,https://github.com/root-project/root/pull/4799,C:\Users\Dmitrijs\Documents\myDocs\masters\cou...,4,237c3105f27dc32284a9c5e910ca6d8f,2
root.root-project.ISSUE_COMMENT.reliability.6bd061ccff03bbbb515b4d8afde33bde.fail.failed,https://github.com/root-project/root/pull/4512:6,reliability,fail,failed,6.0,Build failed because of `Build failed because:...,ISSUE_COMMENT,None,root-project,root,v6-32-06,https://root.cern,https://github.com/root-project/root/pull/4512,C:\Users\Dmitrijs\Documents\myDocs\masters\cou...,4,6bd061ccff03bbbb515b4d8afde33bde,2


In [24]:
# Potentially can be eliminated
df[df.sentence.str.startswith("Starting build")]

,id,quality_attribute,keyword,matched_word,match_idx,sentence,source,filename,author,repo,version,wiki,url,fname,sentence_counts,text_hash,similar_count
real_id,,,,,,,,,,,,,,,,,
root.root-project.ISSUE_COMMENT.deployability.3ebbd896c4b05074fc0eeb4331ecd52a.build.build,https://github.com/root-project/root/pull/5148:9,deployability,build,build,9.0,Starting build on ROOT-performance-centos7-mul...,ISSUE_COMMENT,None,root-project,root,v6-32-06,https://root.cern,https://github.com/root-project/root/pull/5148,C:\Users\Dmitrijs\Documents\myDocs\masters\cou...,1,3ebbd896c4b05074fc0eeb4331ecd52a,1
root.root-project.ISSUE_COMMENT.deployability.416b408c67daa7d82e4333c7743db90a.build.build,https://github.com/root-project/root/pull/5842:9,deployability,build,build,9.0,Starting build on ROOT-performance-centos7-mul...,ISSUE_COMMENT,None,root-project,root,v6-32-06,https://root.cern,https://github.com/root-project/root/pull/5842,C:\Users\Dmitrijs\Documents\myDocs\masters\cou...,1,416b408c67daa7d82e4333c7743db90a,1
root.root-project.ISSUE_COMMENT.deployability.def7f6a0789a4e70ad91a4d4be907a7a.build.build,https://github.com/root-project/root/pull/12171:9,deployability,build,build,9.0,"Starting build on ROOT-ubuntu2204/default, ROO...",ISSUE_COMMENT,None,root-project,root,v6-32-06,https://root.cern,https://github.com/root-project/root/pull/12171,C:\Users\Dmitrijs\Documents\myDocs\masters\cou...,1,def7f6a0789a4e70ad91a4d4be907a7a,1
root.root-project.ISSUE_COMMENT.performance.3ebbd896c4b05074fc0eeb4331ecd52a.perform.performance-,https://github.com/root-project/root/pull/5148:23,performance,perform,performance-,23.0,Starting build on ROOT-performance-centos7-mul...,ISSUE_COMMENT,None,root-project,root,v6-32-06,https://root.cern,https://github.com/root-project/root/pull/5148,C:\Users\Dmitrijs\Documents\myDocs\masters\cou...,2,3ebbd896c4b05074fc0eeb4331ecd52a,1
root.root-project.ISSUE_COMMENT.performance.416b408c67daa7d82e4333c7743db90a.perform.performance-,https://github.com/root-project/root/pull/5842:23,performance,perform,performance-,23.0,Starting build on ROOT-performance-centos7-mul...,ISSUE_COMMENT,None,root-project,root,v6-32-06,https://root.cern,https://github.com/root-project/root/pull/5842,C:\Users\Dmitrijs\Documents\myDocs\masters\cou...,2,416b408c67daa7d82e4333c7743db90a,1
root.root-project.ISSUE_COMMENT.usability.3ebbd896c4b05074fc0eeb4331ecd52a.perform.performance-,https://github.com/root-project/root/pull/5148:23,usability,perform,performance-,23.0,Starting build on ROOT-performance-centos7-mul...,ISSUE_COMMENT,None,root-project,root,v6-32-06,https://root.cern,https://github.com/root-project/root/pull/5148,C:\Users\Dmitrijs\Documents\myDocs\masters\cou...,2,3ebbd896c4b05074fc0eeb4331ecd52a,1
root.root-project.ISSUE_COMMENT.usability.416b408c67daa7d82e4333c7743db90a.perform.performance-,https://github.com/root-project/root/pull/5842:23,usability,perform,performance-,23.0,Starting build on ROOT-performance-centos7-mul...,ISSUE_COMMENT,None,root-project,root,v6-32-06,https://root.cern,https://github.com/root-project/root/pull/5842,C:\Users\Dmitrijs\Documents\myDocs\masters\cou...,2,416b408c67daa7d82e4333c7743db90a,1


In [25]:
df.matched_word = df.matched_word.str.lower()

In [26]:
df[df.qa == "security"].pivot_table(index=["keyword", "matched_word"], columns=["repo_id"],values=["similar_count"], aggfunc="sum", fill_value=0, margins=True).sort_values(by=("similar_count", "All"),ascending=False)

similar_count                                \
repo                              deepvariant fastp    root scanpy scispacy   
keyword matched_word                                                          
All                                      9157  1091  156006   8790     2487   
ident   identifier                          1     4   11178     10       42   
updat   update                            368    99    7687    541      106   
model   model                            1979    11    4806    224      529   
availab available                         289    57    5735    318       90   
...                                       ...   ...     ...    ...      ...   
validat validatefieldparenttype             0     0       1      0        0   
        validateendofmodule                 0     0       1      0        0   
        validateeditrange                   0     0       1      0        0   
ident   identifes                           0     0       1      0        0   
validat validatedeletebb                    0     0       1      0        0   

                                         
repo                                All  
keyword matched_word                     
All                              177531  
ident   identifier                11235  
updat   update                     8801  
model   model                      7549  
availab available                  6489  
...                                 ...  
validat validatefieldparenttype       1  
        validateendofmodule           1  
        validateeditrange             1  
ident   identifes                     1  
validat validatedeletebb              1  

[2447 rows x 6 columns]

In [27]:
df.pivot_table(index=["keyword", "matched_word"], values=["similar_count"], columns=["qa"], aggfunc="sum", fill_value=0, margins=True).sort_values(by=("similar_count", "All"),ascending=False)

similar_count                                  \
quality_attribute      availability deployability energy efficiency   
keyword matched_word                                                  
All                          107837        247938            145907   
error   error                 25334             0                 0   
version version                   0         16986                 0   
modul   module                    0         13607                 0   
test    test                      0             0                 0   
...                             ...           ...               ...   
xml     xmlsaxhandler             0             0                 0   
        xmls                      0             0                 0   
        xmlrpc                    0             0                 0   
        xmlreadfile               0             0                 0   
        xmlerr                    0             0                 0   

                                                                    \
quality_attribute     integrability interoperability modifiability   
keyword matched_word                                                 
All                          176030           142805        168000   
error   error                     0                0             0   
version version               16986                0         16986   
modul   module                    0                0         13607   
test    test                      0                0             0   
...                             ...              ...           ...   
xml     xmlsaxhandler             0                1             0   
        xmls                      0                1             0   
        xmlrpc                    0                1             0   
        xmlreadfile               0                1             0   
        xmlerr                    0                1             0   

                                                                            \
quality_attribute     performance reliability  safety security testability   
keyword matched_word                                                         
All                        163759       43881  240965   177531      143040   
error   error               25334           0   25333        0           0   
version version                 0           0       0        0           0   
modul   module                  0           0   13608        0           0   
test    test                    0           0   20183        0       20174   
...                           ...         ...     ...      ...         ...   
xml     xmlsaxhandler           0           0       0        0           0   
        xmls                    0           0       0        0           0   
        xmlrpc                  0           0       0        0           0   
        xmlreadfile             0           0       0        0           0   
        xmlerr                  0           0       0        0           0   

                                          
quality_attribute     usability      All  
keyword matched_word                      
All                      239433  1997126  
error   error             25334   101335  
version version               0    50958  
modul   module                0    40822  
test    test                  0    40357  
...                         ...      ...  
xml     xmlsaxhandler         0        1  
        xmls                  0        1  
        xmlrpc                0        1  
        xmlreadfile           0        1  
        xmlerr                0        1  

[16669 rows x 13 columns]

In [28]:
df.groupby("keyword").matched_word.nunique().sort_values(ascending=False)

keyword
sub                     1048
test                     987
build                    430
load                     291
modul                    257
                        ... 
applic perform             1
architectur descript       1
ux                         1
adversar                   1
xss                        1
Name: matched_word, Length: 537, dtype: int64

In [29]:
df.groupby(["keyword", "matched_word"]).similar_count.sum()

keyword   matched_word           
abstract  abstract                   3226
          abstract-class                2
          abstract-declarator          76
          abstract-declarators          6
          abstract-interpretation       2
                                     ... 
xml       xmlsoft                       1
          xmlsummary-exception          2
          xmltag                        2
          xmlwriteobject                1
xss       xss-protection                1
Name: similar_count, Length: 16668, dtype: int64

In [30]:
df.head()

,id,quality_attribute,keyword,matched_word,match_idx,sentence,source,filename,author,repo,version,wiki,url,fname,sentence_counts,text_hash,similar_count
real_id,,,,,,,,,,,,,,,,,
deepvariant.google.CODE_COMMENT.availability.00a64ba49a8c8077515638fc173723bc.checkpoint.checkpoint,https://github.com/google/deepvariant/tree/v1....,availability,checkpoint,checkpoint,32.0,"""""""Return the path to the model checkpoint bas...",CODE_COMMENT,scripts/run_deepsomatic.py,google,deepvariant,v1.6.1,None,https://github.com/google/deepvariant/tree/v1....,C:\Users\Dmitrijs\Documents\myDocs\masters\cou...,6,00a64ba49a8c8077515638fc173723bc,3
deepvariant.google.CODE_COMMENT.availability.00bdf0c917332338e88e526ef0912d47.checkpoint.checkpoint,https://github.com/google/deepvariant/tree/v1....,availability,checkpoint,checkpoint,49.0,# If the class variable name doesn't existin t...,CODE_COMMENT,deepvariant/dv_utils_test.py,google,deepvariant,v1.6.1,None,https://github.com/google/deepvariant/tree/v1....,C:\Users\Dmitrijs\Documents\myDocs\masters\cou...,2,00bdf0c917332338e88e526ef0912d47,1
deepvariant.google.CODE_COMMENT.availability.00c0e7cdceae45fdecf2163aa50041d2.incid.incidental,https://github.com/google/deepvariant/tree/v1....,availability,incid,incidental,1092.0,/*. * Copyright 2018 Google LLC. *. * Redistri...,CODE_COMMENT,deepvariant/realigner/fast_pass_aligner.cc,google,deepvariant,v1.6.1,None,https://github.com/google/deepvariant/tree/v1....,C:\Users\Dmitrijs\Documents\myDocs\masters\cou...,46,00c0e7cdceae45fdecf2163aa50041d2,23
deepvariant.google.CODE_COMMENT.availability.00c0e7cdceae45fdecf2163aa50041d2.servic.services,https://github.com/google/deepvariant/tree/v1....,availability,servic,services,1222.0,/*. * Copyright 2018 Google LLC. *. * Redistri...,CODE_COMMENT,deepvariant/realigner/fast_pass_aligner.cc,google,deepvariant,v1.6.1,None,https://github.com/google/deepvariant/tree/v1....,C:\Users\Dmitrijs\Documents\myDocs\masters\cou...,92,00c0e7cdceae45fdecf2163aa50041d2,23
deepvariant.google.CODE_COMMENT.availability.00d2b41f72e591d66c3134b8110fdd4f.operat.operator,https://github.com/google/deepvariant/tree/v1....,availability,operat,operator,12.0,# The match operator = indicates that a positi...,CODE_COMMENT,deepvariant/realigner/window_selector_test.py,google,deepvariant,v1.6.1,None,https://github.com/google/deepvariant/tree/v1....,C:\Users\Dmitrijs\Documents\myDocs\masters\cou...,1,00d2b41f72e591d66c3134b8110fdd4f,1


In [31]:
# check how reduced dataset gets
df.reset_index().groupby(["sentence"]).agg({'similar_count': 'sum', 'real_id': 'count'}).nlargest(1000, 'similar_count')

,similar_count,real_id
sentence,,
@phsft-bot build,2314,1
"# Copyright 2018 Google LLC. #. # Redistribution and use in source and binary forms, with or without. # modification, are permitted provided that the following conditions. # are met:. #. # 1. Redistributions of source code must retain the above copyright notice,. # this list of conditions and the following disclaimer. #. # 2. Redistributions in binary form must reproduce the above copyright. # notice, this list of conditions and the following disclaimer in the. # documentation and/or other materials provided with the distribution. #. # 3. Neither the name of the copyright holder nor the names of its. # contributors may be used to endorse or promote products derived from this. # software without specific prior written permission. #. # THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS ""AS IS"". # AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE. # IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE. # ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE. # LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR. # CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF. # SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS. # INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN. # CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE). # ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE. # POSSIBILITY OF SUCH DAMAGE.",1200,16
@phsft-bot build!,1124,1
"# Copyright 2017 Google LLC. #. # Redistribution and use in source and binary forms, with or without. # modification, are permitted provided that the following conditions. # are met:. #. # 1. Redistributions of source code must retain the above copyright notice,. # this list of conditions and the following disclaimer. #. # 2. Redistributions in binary form must reproduce the above copyright. # notice, this list of conditions and the following disclaimer in the. # documentation and/or other materials provided with the distribution. #. # 3. Neither the name of the copyright holder nor the names of its. # contributors may be used to endorse or promote products derived from this. # software without specific prior written permission. #. # THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS ""AS IS"". # AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE. # IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE. # ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE. # LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR. # CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF. # SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS. # INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN. # CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE). # ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE. # POSSIBILITY OF SUCH DAMAGE.",864,16
"/*. * Copyright 2018 Google LLC. *. * Redistribution and use in source and binary forms, with or without. * modification, are permitted provided that the following conditions. * are met:. *. * 1. Redistributions of source code must retain the above copyright notice,. * this list of conditions and the following disclaimer. *. * 2. Redistributions in binary form must reproduce the above copyright. * notice, this list of conditions and the following disclaimer in the. * documentation and/or other materials provided with the distribution. *. * 3. Neither the name of the copyright holder nor the names of its. * contributors may be used to endorse or promote products derived from this. * software without specific prior written permission. *. * THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS ""AS IS"". * AND 

['google.deepvariant.v1.6.1',
 'OpenGene.fastp.v0.23.4',
 'root-project.root.v6-32-06',
 'scverse.scanpy.1.10.2',
 'allenai.scispacy.v0.5.5']

In [33]:
df.groupby("keyword").matched_word.agg(['unique','nunique']).sort_values(by="nunique", ascending=False)

,unique,nunique
keyword,,
sub,"[subprocess, substitute, subsequent, substitut...",1048
test,"[test, testing, tests, testing-cabal, tested, ...",987
build,"[builds, build, building, build-test, build-pr...",430
load,"[load, loading, loaded, loads, loader, loadcpu...",291
modul,"[modules, module-level, module, modulenotfound...",257
...,...,...
applic perform,[application performs],1
architectur descript,[architecture description],1
ux,[ux],1


In [34]:
df.groupby(["qa", "keyword"]).matched_word.agg(['unique','nunique']).sort_values(by="nunique", ascending=False)

,,unique,nunique
quality_attribute,keyword,,
integrability,sub,"[subprocess, substitute, subsequent, substitut...",1048
safety,test,"[test, testing, tests, testing-cabal, tested, ...",987
testability,test,"[test, testing, tests, testing-cabal, tested, ...",981
deployability,build,"[builds, build, building, build-test, build-pr...",430
energy efficiency,load,"[load, loading, loaded, loads, loader, loadcpu...",291
...,...,...,...
availability,slo,[slo],1
usability,mock-up,[mock-up],1
availability,watchdog,[watchdog],1


In [35]:
df.groupby(["keyword", "qa"]).matched_word.agg(['unique','nunique'])

unique  \
keyword  quality_attribute                                                      
abstract integrability      [abstract, abstraction, abstractmethods, abstr...   
         modifiability      [abstract, abstraction, abstractmethods, abstr...   
access   security           [access, accessed, accesses, accessor, accessi...   
accid    safety             [accidentally, accident, accidents, accidental...   
adapt    energy efficiency  [adapted, adapter, adaptor, adapters, adapts, ...   
...                                                                       ...   
worm     security                                           [wormbase, worms]   
wrap     integrability      [wrapper, wrapped, wrapping, wrap, wrappers, w...   
wrapper  interoperability   [wrapper, wrappers, wrapperfrontendaction, wra...   
xml      interoperability   [xmlns, xml, xmlnode, xml-like, xmlparser, xml...   
xss      security                                            [xss-protection]   

                            nunique  
keyword  quality_attribute           
abstract integrability           46  
         modifiability           46  
access   security                89  
accid    safety                   7  
adapt    energy efficiency       37  
...                             ...  
worm     security                 2  
wrap     integrability           70  
wrapper  interoperability        25  
xml      interoperability        51  
xss      security                 1  

[686 rows x 2 columns]

In [42]:
df.groupby(["keyword", "qa", "matched_word"])['sentence'].apply(lambda x: x.head(3).to_list())

keyword   quality_attribute  matched_word           
abstract  integrability      abstract                   [// Alias for the abstract base class for BED ...
                             abstract-class             [/// Indicates that the definition of this cla...
                             abstract-declarator        [// If we haven't past the identifier yet (or ...
                             abstract-declarators       [/// isFunctionDeclaratorIdentifierList - This...
                             abstract-interpretation    [ion by representing unknown input values symb...
                                                                              ...                        
xml       interoperability   xmlsoft                    [BEGIN_HTML. This directory contains classes t...
                             xmlsummary-exception       [cling crash when moving to LCG103 in LHCb; ##...
                             xmltag                     [constant<...>' for 2nd argument. void _M_r

In [36]:
df.groupby(["keyword", "qa"]).agg(
    matched_words=("matched_word", 'unique'),
    nunique=("matched_word", 'nunique'),
    total_matched=("similar_count", "sum")
)

matched_words  \
keyword  quality_attribute                                                      
abstract integrability      [abstract, abstraction, abstractmethods, abstr...   
         modifiability      [abstract, abstraction, abstractmethods, abstr...   
access   security           [access, accessed, accesses, accessor, accessi...   
accid    safety             [accidentally, accident, accidents, accidental...   
adapt    energy efficiency  [adapted, adapter, adaptor, adapters, adapts, ...   
...                                                                       ...   
worm     security                                           [wormbase, worms]   
wrap     integrability      [wrapper, wrapped, wrapping, wrap, wrappers, w...   
wrapper  interoperability   [wrapper, wrappers, wrapperfrontendaction, wra...   
xml      interoperability   [xmlns, xml, xmlnode, xml-like, xmlparser, xml...   
xss      security                                            [xss-protection]   

                            nunique  total_matched  
keyword  quality_attribute                          
abstract integrability           46           2404  
         modifiability           46           2404  
access   security                89           9954  
accid    safety                   7            312  
adapt    energy efficiency       37           1960  
...                             ...            ...  
worm     security                 2              7  
wrap     integrability           70           4531  
wrapper  interoperability        25           2313  
xml      interoperability        51           1715  
xss      security                 1              1  

[686 rows x 3 columns]

In [ ]:
df.groupby(["keyword", "qa"]).agg(
    matched_words=("matched_word", 'unique'),
    nunique=("matched_word", 'nunique'),
    total_matched=("similar_count", "sum")
).reset_index().to_csv(AbsDirPath.KEYWORD_ANALYSIS / "all_matched_words_by_keywords_&_QAs.csv", index=False, header=True)

In [ ]:
import itertools

sources = pd.Series(df.source.unique()).str.split(".").str[0].tolist()
repos = df.repo_id.unique()
output_dir = AbsDirPath.OPTIMIZED_KEYWORDS
output_dir.mkdir(exist_ok=True)

for source, repo_id in itertools.product(sources, repos):
    output_file = output_dir / f"{repo_id}.{source}.parquet"
    print(output_file)
    df[(df.source == source) & (df.repo_id == repo_id)].drop(columns=["id", "fname", "sentence_counts"]).to_parquet(output_file, engine='pyarrow', compression='snappy', index=False)
    print(f"Saved {output_file}")

In [ ]:
df